In [ ]:
# IGNORE THIS CELL WHICH CUSTOMIZES LAYOUT AND STYLING OF THE NOTEBOOK !
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings = lambda *a, **kw: None

# Exercise 3_psyplot Mesh plots (psyplot)
prepared by A. Lauber

Here we learn how to plot data from an unstructured grid like the ICON model as mesh grid.
There is the option to interpolate the data to a regular grid or to use the library [psyplot](https://psyplot.github.io), which was developed for plotting data on the unstructured grid.

Advantages of plotting on the unstructured grid:
- Shows the real output and no interpolation
- No preprocessing of the data necessary

Disadvantages of plotting on the unstructured grid:
- It can be quite slow depending on the size of your netCDF file
- Not all features are available yet (psyplot is still under development)


## Import libraries

In [ ]:
import xarray as xr
import numpy as np
import cartopy.crs as ccrs
import cartopy
import matplotlib.pyplot as plt
import cmcrameri.cm as cmc
import psyplot.project as psy
import urllib.request
from cdo import *

psy.rcParams['auto_show'] = True

In [ ]:
#Download data from ftp server
urllib.request.urlretrieve("ftp://iacftp.ethz.ch/pub_read/alauber/nc/my_exp1_atm_3d_ml_20180921T000000Z.nc", "my_exp1_atm_3d_ml_20180921T000000Z.nc")

In [ ]:
# Have a look into the data
ds = xr.open_dataset("my_exp1_atm_3d_ml_20180921T000000Z.nc")
np.min(ds['clon'].values)

In [ ]:
# Try to plot t_2m with pcolormesh.
# Why does it not work?
lon, lat, temp = ds.clon, ds.clat, ds.t_2m[:,:,:]

ax = plt.axes(projection=ccrs.Robinson())

h = ax.pcolormesh(lon,lat,temp, transform=ccrs.PlateCarree(),cmap = cmc.nuuk,)

In [ ]:
# Download the same data but a remapped version
urllib.request.urlretrieve("ftp://iacftp.ethz.ch/pub_read/alauber/pyvis/rmp_my_exp1_atm_3d_ml_20180921T000000Z.nc", "rmp_my_exp1_atm_3d_ml_20180921T000000Z.nc")

In [ ]:
# Have a look into the data
ds_rmp = xr.open_dataset("rmp_my_exp1_atm_3d_ml_20180921T000000Z.nc")
ds_rmp

In [ ]:
# get data
lon, lat, temp = ds_rmp.lon, ds_rmp.lat, ds_rmp.t_2m[0,0,:,:]

ax = plt.axes(projection=ccrs.Robinson())

h = ax.pcolormesh(lon,lat,temp, transform=ccrs.PlateCarree(),shading='nearest',cmap = cmc.nuuk,
             vmin=260,vmax=300)

ax.add_feature(cartopy.feature.BORDERS)

ax.set_extent([5.8, 10.7, 45.5, 48])
plt.colorbar(h)
plt.show()

In [ ]:
ds_icon = psy.open_dataset("../../icon-vis/data/example_data/nc/my_exp1_atm_3d_ml_20180921T000000Z.nc")
ds_icon

In [ ]:
%config InlineBackend.close_figures = False
import iconarray
lonmin = 5.8
lonmax = 10.7
latmin = 45.5
latmax = 48.0
plot_icon = ds_icon.psy.plot.mapplot(
    name = 't_2m',
    t = 0,
    projection = 'robin',
    bounds = {'method': 'minmax', 'vmin':260, 'vmax':300},
    map_extent = [lonmin, lonmax, latmin, latmax],
    title = '2m temperature',
    cmap = cmc.nuuk,
    borders=True,
    xgrid=False,
    ygrid=False)

In [ ]:
ds_alnu = xr.open_dataset("../data.nc")
ds_alnu.ALNU.values = np.log10(ds_alnu.ALNU.values + 1)
ds_alnu

In [ ]:
# Checkout the formatoptions (https://psyplot.github.io/psy-maps/generated/psyplot.project.plot.mapplot.html)
# change the grid color to white, add a label to the colorbar, change the projection and cmap etc.
plot_alnu = ds_alnu.psy.plot.mapplot(
    name="ALNU",
    title="Alder Pollen in the Alps on 2020-02-15 20h",
    titlesize=15,
    grid_color="white",
    cticksize=8,
    clabel="Log. Pollen Concentrations [Pollen/m³]",
    grid_labelsize=8,
    projection='robin',
    cmap='RdBu_r')


In [ ]:
#Checkout https://github.com/C2SM/iconarray
import iconarray 

In [ ]:
plot_alnu = ds_alnu.psy.plot.mapplot(
    name="ALNU",
    title="Alder Pollen in the Alps on 2020-02-15 20h",
    titlesize=15,
    grid_color="white",
    cticksize=8,
    clabel="Log. Pollen Concentrations [Pollen/m³]",
    grid_labelsize=8,
    projection='robin',
    cmap='RdBu_r')


In [ ]:
plot_alnu.update(lakes=False, borders=False, rivers=False)